# Pathlib Challenges

For these challenges we will be getting familiar with the `pathlib` library.

This topic was inspired by our own Organizer, Chris May. He wrote an article on [getting started with pathlib](https://everydaysuperpowers.dev/articles/stop-working-so-hard-paths-get-started-pathlib/) which also prompted him to write a [field guide](https://everydaysuperpowers.dev/documents/3/ES-Getting_Started_with_Pathlib.pdf) and [cheat sheet](https://everydaysuperpowers.dev/documents/2/pathlib_cheat_sheet-V1_200703.pdf). You can see his other articles and resources at https://everydaysuperpowers.dev. Feel free to look at the resources and the `pathlib` [documentation](https://docs.python.org/3/library/pathlib.html) as they serve as the basis for the exercises below.

There are two kinds of Paths. The documentation states, "*Path classes are divided between pure paths, which provide purely computational operations without I/O, and concrete paths, which inherit from pure paths but also provide I/O operations.*" In other words,

* **`PurePath`**: Performs path operations witout caring about what might actually be on the disk.
* **`Path`**: Allows you to interact with files.

Both `PurePath` and `Path` can be either Windows or Posix paths. Chances are that you will not need to worry about the operating system specific choice since `pathlib` will take care of all the specifics for you!

One common shortcut to get the path of the current file is with the `__file__` variable. Due to how Jupyter Notebooks work, the `__file__` variable is not available, so it is mimmicked in the imports block. If you're not familiar with `__file__`, I suggest you look at [`__name__`](https://docs.python.org/3/library/__main__.html) and how python uses [dunder](https://dbader.org/blog/python-dunder-methods) attributes and methods.

In [1]:
import json
import os
import string
from pathlib import Path
from exercise import setup, cleanup

setup()
__file__ = os.path.join(os.getcwd(), 'Challenge.ipynb')
__file__

 12%|█▎        | 25/200 [00:06<00:48,  3.63it/s]


KeyboardInterrupt: 

# Where am I?

The Zen of Python states

    There should be one-- and preferably only one --obvious way to do it.
    Although that way may not be obvious at first unless you're Dutch.
    
So naturally it would follow that there are many ways to access the current directory as a place to start. There are 11 different methods shown below for gettinging the current directory. Take some time and look over how each line works. Some things to note:

* `os.getcwd()` is shown, but only included for comparison purposes.
* In all major operating systems `.` refers to the current directory and `..` referrs to the directory above.
* Note the difference between lines with and without `.resolve()`
* Note how `__file__` requires the `parent` attribute.
* Note the difference between `.parent` and `.parents`.

While these are all interchangable here, they are not always interchangeable. Some will refer to the working directory while others refer to the location of the script. Be careful with which one you choose. 

In [ ]:
[
    os.getcwd(),
    Path(os.getcwd()),
    Path.cwd(),
    Path(),
    Path().resolve(),
    Path('.'),
    Path('.').resolve(),
    Path(__file__) / '..',
    (Path(__file__) / '..').resolve(),
    Path(__file__).parent,
    Path(__file__).parents[0],
]

Okay, now that you've looked at the different ways of accessing the current directory, use one of them to set a variable `base_dir` to the base of the repo (one parent up)

In [ ]:
base_dir = Path().resolve().parent

How many directories are we away from the file system root? What are they?

In [ ]:
[*base_dir.resolve().parents]

# Who am I?

Often, you might want to access the home directory of the user. Create a variable `home_dir` that refers to the current user's home directory (In windows, this is `C:\Users\<user>`, in Linux it's usually `/home/<user>`). Does the Path look right? If you launched this notebook by clicking the Binder link in the repo, [this](https://github.com/jupyter/docker-stacks/issues/358#issuecomment-288844834) might answer a question you're now asking.

**You should not be hard coding the file path.**

In [ ]:
home_dir = Path.home()
home_dir

# What Have We Got Here?

Now that you know where you are, it's time to see what is there with you.

List all the items in the home directory.

In [ ]:
[f for f in home_dir.iterdir()]

Only list items in the home directory that are files.

In [ ]:
[f for f in home_dir.iterdir() if f.is_file()]

Only list items in the home directory that are directories.

In [ ]:
[d for d in home_dir.iterdir() if d.is_dir()]

Find all the notebooks under `base_dir` (`.ipynb`)

In [ ]:
[f for f in base_dir.rglob('*.ipynb')]

What versions of python are installed on this machine? (If you're running linux or mac, look for files containing `python` under `/usr/bin`. If using windows, skip this and the next challenge.)

In [ ]:
[f for f in Path('/usr/bin').glob('python*')]

Which python files are just symbolic links to actual files?

In [ ]:
[f for f in Path('/usr/bin').glob('python*') if f.is_symlink()]

Print the contents of `requirements.txt` under `base_dir`. Do this with one line.

In [ ]:
print(base_dir.joinpath('requirements.txt').read_text())

# File Properties

In [ ]:
# year_ago = datetime.datetime.now() - datetime.timedelta(days=365)

# for f in Path.home().iterdir():
#     try:
#         mtime = datetime.datetime.fromtimestamp(f.stat().st_mtime)
#         if mtime < year_ago:
#             print(f, mtime)
#     except FileNotFoundError:
#         pass

# Rename Files

The `setup()` function at the top of the notebook downloaded images from the first 10 pages from http://books.toscrape.com/ and compiled the data into a json file.

The json file and images are located in a directory called `books`. List all the images in the directory. (They are all jpg)

In [ ]:
[f for f in Path('books').rglob('*.jpg')]

Can you tell the titles of the different books? If not, load the json file and try to rename each file.

In [ ]:
data = json.loads(list(Path('books').rglob('*.json'))[0].read_text())
mapping = {Path(book['img']).stem: book['name'] for book in data}

for img in Path('books').rglob('*.jpg'):
    new_name = img.with_name(mapping[img.stem] + img.suffix)
    img.rename(new_name)    

Now check to see that you can figure out which image is which. (copy your code from a few cells up.)

In [ ]:
[f for f in Path('books').rglob('*.jpg')]

# Let's Make Something

Under `base_dir`, Create a path to a file called `example.txt`.

In [ ]:
file = base_dir / 'example.txt'

Check to see if the file exists

In [ ]:
file.exists()

Write text to the file using one of the `pathlib` utility methods.

In [ ]:
file.write_text('PyRVA')

See if the file exits now.

In [ ]:
file.exists()

Get the text from the file using one of the `pathlib` utility methods.

In [ ]:
file.read_text()

Under `base_dir`:

* Create a new directory called `new`. 
* In `new`, create 26 sub directories based on the letters of the alphabet (you can use `string.ascii_lowercase` to iterate if you'd like). 
* Under each letter directory, create 10 directories numbered `0` -> `9` (you can use `range(10)` to iterate if you'd like.) 
* In each numbered directory, create a file called `file.txt` and write `PyRVA is Awesome!` to each file.

You should only have one line in the code block that actually creates directories and one line in the code block that creates the file.

What happens if you run the code block twice?

In [ ]:
for char in string.ascii_lowercase:
    for i in range(10):
        new = base_dir / 'new' / char / str(i)
        new.mkdir(parents=True)        
        (new / 'file.txt').write_text('PyRVA is Awesome!')

Find all the `file.txt` files under `base_dir` and check that the number matches what you expected (this can be done in one line). If not, you might need to debug something.

In [ ]:
len([*base_dir.rglob('file.txt')])

In [ ]:
[f for f in base_dir.rglob('file.txt') if 'new' not in str(f)]

Now delete the `new` folder.

In [ ]:
def rmdir(top: Path):
    for file in top.iterdir():
        if file.is_file():
            file.unlink()
        elif file.is_dir():
            rmdir(file)
    top.rmdir()
            
rmdir(base_dir / 'new')

# Which Is Better?

For these exercises, see if you can figure out the pathlib replacement for the following code blocks. There is a quick reference table at the bottom of the `pathlib` documentation, but I encourage you to read through the documentation without looking at the table. Once you have verified the output to be the same (don't worry about the `PosixPath()` `__repr__` value, as long as the result is the same otherwise, you can use `str()` if you really care about it), use the [`%%timeit`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit) magic method to compare the speeds.

Once you have compared the different methods, think about which of the follow things are important to you:

* Speed of development (characters typed)
* Speed of maintenance (how easy is it to read)
* Speed of execution (how fast the code runs)

Getting the parent of the current folder.

In [ ]:
os.path.dirname(os.path.dirname(os.path.abspath(__file__)))

In [ ]:
Path(__file__).parents[1]

Get the basename of the current file.

In [ ]:
os.path.basename(__file__)

In [ ]:
Path(__file__).name

Get the basename of the current file without the extension.

In [ ]:
os.path.splitext(os.path.basename(__file__))[0]

In [ ]:
Path(__file__).stem

Get the file extension of the current file.

In [ ]:
os.path.splitext(__file__)[1]

In [ ]:
Path(__file__).suffix

Create a `PurePath` with a different name in the same directory (file doesn't have to exist).

In [ ]:
os.path.join(os.path.dirname(__file__), 'myfile.txt')

In [ ]:
Path(__file__).with_name('myfile.txt')

Create a `PurePath` of the same file with a different extension.

In [ ]:
os.path.join(os.path.dirname(__file__), os.path.basename(os.path.splitext(__file__)[0]) + '.py')

In [ ]:
Path(__file__).with_suffix('.py')

Get the home directory of the current user.

In [ ]:
os.path.expanduser('~')

In [ ]:
Path.home()

# Cleanup

Run the following line to remove files created by this exercise (assuming you followed the suggested file names).

In [ ]:
cleanup()